<a href="https://colab.research.google.com/github/dkmau2004/Content/blob/main/Autoimmunity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Drug Induced Autoimmunity

In [ ]:
pip install scikit-optimize

##Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from collections import Counter

import xgboost as xgb
from xgboost import XGBRegressor, plot_importance, XGBClassifier

from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from skopt import BayesSearchCV
from skopt.space import Real, Integer

from scipy.stats import pointbiserialr

import shap

##Reading Files

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

##Analysing the Data

In [ ]:
print("Dataset of train:")
display(train_df) #Showing the contents of the datasheet

In [ ]:
print("Dataset of test:")
display(test_df)

In [ ]:
plt.figure(figsize=(9, 8))
sns.countplot(data=train_df, x="Label", stat="percent", color="steelblue")
plt.ylabel("Percentage (%)")
plt.xlabel("Label")
plt.title("Label Distribution in %")
plt.show()

###Correlation Analysis

In [ ]:
# Assume train_df contains all descriptors + 'Label'
corr = train_df.drop(columns=["SMILES", "Label"]).corr()
plt.figure(figsize=(15, 12))
sns.heatmap(corr, cmap="RdBu_r", center=0, vmax=1, vmin=-1)
plt.title("Feature-to-Feature Correlation Heatmap")
plt.show()

In [ ]:
dropped_due_to_high_correlations = ["Chi0n", "Chi0v", "Chi1", "Chi1n", "Chi1v", "NumSaturatedCarbocycles", "fr_Nhpyrrole", "MinAbsPartialCharge", "Chi2v", "Chi2n", "Chi3n", "MaxEStateIndex", "Chi3v", "Chi4n", "EState_VSA10", "Chi4v", "NumValenceElectrons", "SMR_VSA1", "VSA_EState9", "MolWt", "MolMR", "LabuteASA", "Kappa1", "HeavyAtomMolWt", "HeavyAtomCount", "ExactMolWt"]

In [ ]:
cols_to_drop = ["SMILES"] + dropped_due_to_high_correlations

In [ ]:
#Removing some of the highly correlated.
corr = train_df.drop(columns=["SMILES", "Label", "Chi0n", "Chi0v", "Chi1", "Chi1n", "Chi1v", "Chi2v", "fr_Nhpyrrole", "MaxEStateIndex", "Chi2n", "MinAbsPartialCharge", "EState_VSA10", "Chi3n", "Chi3v", "Chi4n", "Chi4v", "NumValenceElectrons", "SMR_VSA1", "VSA_EState9", "MolWt", "MolMR", "LabuteASA", "Kappa1", "HeavyAtomMolWt", "HeavyAtomCount", "ExactMolWt"]).corr()
plt.figure(figsize=(15, 12))
sns.heatmap(corr, cmap="RdBu_r", center=0, vmax=1, vmin=-1)
plt.title("Feature-to-Feature Correlation Heatmap")
plt.show()

In [ ]:
corr.corr().style.background_gradient(cmap='viridis')

In [ ]:
corr2 = train_df.drop(columns=["SMILES", "Label", "Chi0n", "Chi0v", "Chi1", "Chi1n", "Chi1v", "fr_Nhpyrrole", "Chi2v", "NumSaturatedCarbocycles", "MaxEStateIndex", "Chi2n", "MinAbsPartialCharge", "EState_VSA10", "Chi3n", "Chi3v", "Chi4n", "Chi4v", "NumValenceElectrons", "SMR_VSA1", "VSA_EState9", "MolWt", "MolMR", "LabuteASA", "Kappa1", "HeavyAtomMolWt", "HeavyAtomCount", "ExactMolWt"]).corr()

In [ ]:
corr2.corr().style.background_gradient(cmap='viridis')

In [ ]:
# Correlation with the binary target
feature_cols = train_df.columns.difference(["SMILES", "Label"])
corr_target = {col: pointbiserialr(train_df[col], train_df["Label"])[0] for col in feature_cols}
corr_series = pd.Series(corr_target).sort_values(key=lambda x: abs(x), ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=corr_series.index[:15], y=corr_series.values[:15])
plt.xticks(rotation=90)
plt.title("Top 15 Features by Correlation with Autoimmunity Label")
plt.xlabel("Features")
plt.ylabel("Correlation")
plt.show()


In [ ]:
# Compute absolute point-biserial correlation
corr_target = {
    col: abs(pointbiserialr(train_df[col], train_df["Label"])[0])
    for col in feature_cols
}

# Sort by strongest relationship
corr_series = pd.Series(corr_target).sort_values(ascending=False)

# Top 20 features
plt.figure(figsize=(10, 5))
sns.barplot(x=corr_series.index[:15], y=corr_series.values[:15])
plt.xticks(rotation=90)
plt.title("Top 15 Features by Absolute Correlation with Autoimmunity Label")
plt.xlabel("Features")
plt.ylabel("Absolute Correlation")
plt.show()

This show that fr_aniline has the greatest correlation with drug induced autoimmunity.

### Distribution of some features

In [ ]:
plt.figure(figsize=(10, 5))
plt.xlabel("Label")
plt.ylabel("Percent %")
plt.title("Distribution of Label")
sns.countplot(data=train_df,x='Label', stat= 'percent')
plt.show()

There's more 0 than 1 therefore the model needs to be imputed with data that has 1 so that the model wouldn't prioritise 0 over 1.

In [ ]:
plt.figure(figsize=(10, 5))
plt.xlabel("fr_aniline")
plt.ylabel("Percent %")
plt.title("Distribution of fr_aniline")
sns.countplot(data=train_df,x='fr_aniline', stat= 'percent')
plt.show()

In [ ]:
# Count plot for fr_aniline
plt.figure(figsize=(10, 5))
sns.countplot(data=train_df,x='fr_aniline',hue='Label', stat='percent')
plt.xlabel("fr_aniline")
plt.ylabel("Percent %")
plt.title("Distribution of fr_aniline by DIA Label")
plt.show()

In [ ]:
# Count plot for fr_ArN
plt.figure(figsize=(10, 5))
sns.countplot(data=train_df,x='fr_ArN',hue='Label', stat='percent')
plt.xlabel("fr_ArN")
plt.ylabel("Percent %")
plt.title("Distribution of fr_ArN by DIA Label")
plt.show()

In [ ]:
# Count plot for SlogP_VSA10
plt.figure(figsize=(25, 7))
sns.countplot(data=train_df,x='SlogP_VSA10',hue='Label', stat='percent')
plt.xlabel("SlogP_VSA10")
plt.ylabel("Percent %")
plt.title("Distribution of SlogP_VSA10 by DIA Label")
plt.show()

In [ ]:
# Count plot for fr_amide
plt.figure(figsize=(10, 5))
sns.countplot(data=train_df,x='fr_amide',hue='Label', stat='percent')
plt.xlabel("fr_amide")
plt.ylabel("Percent %")
plt.title("Distribution of fr_amide by DIA Label")
plt.show()

## XGBoost Modelling

In [ ]:
# Separate features and labels
train_df= train_df.drop(columns= cols_to_drop)
test_df= test_df.drop(columns= cols_to_drop)
X_train = train_df.drop("Label", axis=1)
y_train = train_df["Label"]

X_test = test_df.drop("Label", axis=1)
y_test = test_df["Label"]

### SMOTE

In [ ]:

# Before SMOTE
print("Original class distribution:", Counter(y_train))

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# After SMOTE
print("Resampled class distribution:", Counter(y_resampled))

In [ ]:
# xgb_param_grid = {
#     'learning_rate': Real(0.01, 0.2, prior='uniform'),
#     'n_estimators': Integer(100, 300),
#     'max_depth': Integer(1, 10),
#     'min_child_weight': Integer(1, 2),
#     'subsample': Real(0.1, 0.3, prior='uniform'),
#     'colsample_bytree': Real(0.6, 0.9, prior='uniform'),
#     'gamma': Real(0.01, 1, prior='uniform'),
#     'scale_pos_weight': Integer(0, 3)
# }

# xgb_class = BayesSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss"),
#                             search_spaces=xgb_param_grid,
#                             n_iter=150,
#                             cv=5,
#                             verbose=True,
#                             n_jobs=-1,
#                             random_state=42)


# xgb_class.fit(X_resampled, y_resampled)

### Hyperparameter Tuning

In [ ]:
xgb_param_grid = {
    'learning_rate': Real(0.01, 0.3, prior='uniform'),
    'n_estimators': Integer(20, 80),
    'max_depth': Integer(3, 7),
    'min_child_weight': Integer(1, 3),
    'lambda': Real(0.01, 10, prior='uniform'),
    'alpha': Real(0.01, 10, prior='uniform'),
    #'subsample': Real(0.7, 0.95, prior='uniform'),
    #'colsample_bytree': Real(0.6, 0.9, prior='uniform'),
    #'gamma': Real(0.01, 2, prior='uniform'),
    #'scale_pos_weight': Integer(0, 3)
}

xgb_class = BayesSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss", subsample=0.9, gamma=0.5),
                            search_spaces=xgb_param_grid,
                            n_iter=100,
                            cv=5,
                            verbose=True,
                            n_jobs=-1,
                            random_state=42)


xgb_class.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [ ]:
# Get the best hyperparameters from RandomizedSearchCV for XGBoost
best_params_xgb = xgb_class.best_params_

print("Best Hyperparameters for XGBoost:")
for param, value in best_params_xgb.items():
    print(f"  {param}: {value}")

### Predictions

In [ ]:
# Predict on the test set
y_pred = xgb_class.predict(X_test)

# Evaluate performance
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))

# Extra: Detailed report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Extra: Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

In [ ]:
from sklearn.metrics import roc_auc_score

# Compute AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)


In [ ]:
from sklearn.metrics import matthews_corrcoef

mcc = matthews_corrcoef(y_test, y_pred)
print("MCC:", mcc)
